### Extraction Phase

In [2]:
# Installing Necessary Libraries
import os
import io
%pip install pyarrow
%pip install azure-storage-blob
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Importing Necessary Packages
import pandas as pd
import os
import io
from azure.storage.blob import BlobServiceClient, BlobClient
from dotenv import load_dotenv

In [4]:
# Reading the Data from the source
nycpayroll2020_df = pd.read_csv(r'Data\nycpayroll_2020.csv')

In [5]:
nycpayroll2020_df.head()

,FiscalYear,PayrollNumber,AgencyID,AgencyName,EmployeeID,LastName,FirstName,AgencyStartDate,WorkLocationBorough,TitleCode,TitleDescription,LeaveStatusasofJune30,BaseSalary,PayBasis,RegularHours,RegularGrossPaid,OTHours,TotalOTPaid,TotalOtherPay
0,2020,17,2120,OFFICE OF EMERGENCY MANAGEMENT,10001,GEAGER,VERONICA,9/12/2016,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,per Annum,1820.0,84698.21,0.0,0.0,0.00
1,2020,17,2120,OFFICE OF EMERGENCY MANAGEMENT,149612,ROTTA,JONATHAN,9/16/2013,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,per Annum,1820.0,84698.21,0.0,0.0,0.00
2,2020,17,2120,OFFICE OF EMERGENCY MANAGEMENT,206583,WILSON II,ROBERT,4/30/2018,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,per Annum,1820.0,84698.21,0.0,0.0,0.00
3,2020,17,2120,OFFICE OF EMERGENCY MANAGEMENT,199874,WASHINGTON,MORIAH,3/18/2019,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,per Annum,1820.0,87900.95,0.0,0.0,-3202.74
4,2020,17,2120,OFFICE OF EMERGENCY MANAGEMENT,58036,KRAWCZYK,AMANDA,5/15/2017,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,per Annum,1820.0,83976.54,0.0,0.0,0.00


In [6]:
nycpayroll2020_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   FiscalYear             100 non-null    int64  
 1   PayrollNumber          100 non-null    int64  
 2   AgencyID               100 non-null    int64  
 3   AgencyName             100 non-null    object 
 4   EmployeeID             100 non-null    int64  
 5   LastName               100 non-null    object 
 6   FirstName              100 non-null    object 
 7   AgencyStartDate        100 non-null    object 
 8   WorkLocationBorough    100 non-null    object 
 9   TitleCode              100 non-null    int64  
 10  TitleDescription       100 non-null    object 
 11  LeaveStatusasofJune30  100 non-null    object 
 12  BaseSalary             100 non-null    float64
 13  PayBasis               100 non-null    object 
 14  RegularHours           100 non-null    float64
 15  Regular

In [7]:
 # Converting the FiscalYear to datetime
nycpayroll2020_df['FiscalYear'] = pd.to_datetime(nycpayroll2020_df['FiscalYear'], format='%Y') 

In [8]:
nycpayroll2020_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   FiscalYear             100 non-null    datetime64[ns]
 1   PayrollNumber          100 non-null    int64         
 2   AgencyID               100 non-null    int64         
 3   AgencyName             100 non-null    object        
 4   EmployeeID             100 non-null    int64         
 5   LastName               100 non-null    object        
 6   FirstName              100 non-null    object        
 7   AgencyStartDate        100 non-null    object        
 8   WorkLocationBorough    100 non-null    object        
 9   TitleCode              100 non-null    int64         
 10  TitleDescription       100 non-null    object        
 11  LeaveStatusasofJune30  100 non-null    object        
 12  BaseSalary             100 non-null    float64       
 13  PayBas

In [9]:
nycpayroll2021_df = pd.read_csv(r'Data\nycpayroll_2021.csv')

In [10]:
nycpayroll2021_df.head()

,FiscalYear,PayrollNumber,AgencyCode,AgencyName,EmployeeID,LastName,FirstName,AgencyStartDate,WorkLocationBorough,TitleCode,TitleDescription,LeaveStatusasofJune30,BaseSalary,PayBasis,RegularHours,RegularGrossPaid,OTHours,TotalOTPaid,TotalOtherPay
0,2021,996,2153,NYC HOUSING AUTHORITY,209184,MUSTACIUOLO,VITO,2/26/2018,MANHATTAN,40475,EXECUTIVE DIRECTOR,ACTIVE,258000.0,per Annum,1820,257260.30,0.00,0.00,258000.00
1,2021,996,2153,NYC HOUSING AUTHORITY,302330,RUSS,GREGORY,8/12/2019,MANHATTAN,41143,CHAIR,ACTIVE,414707.0,per Annum,1820,413518.05,0.00,0.00,500.00
2,2021,816,2129,DEPT OF HEALTH/MENTAL HYGIENE,49788,HALLAHAN,PATRICK,2/26/2018,BROOKLYN,40782,STATIONARY ENGINEER,ACTIVE,508.8,per Day,2080,132288.00,2115.25,218628.18,56616.07
3,2021,816,2129,DEPT OF HEALTH/MENTAL HYGIENE,251626,PETTIT,PATRICK,8/2/2010,MANHATTAN,40782,STATIONARY ENGINEER,ACTIVE,508.8,per Day,2080,132288.00,2152.75,218694.96,38611.82
4,2021,816,2129,DEPT OF HEALTH/MENTAL HYGIENE,364376,TELEHANY,STEPHEN,1/16/2007,QUEENS,40782,STATIONARY ENGINEER,ACTIVE,508.8,per Day,2080,132288.00,1876.25,192296.19,51160.20


In [11]:
nycpayroll2021_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   FiscalYear             101 non-null    int64  
 1   PayrollNumber          101 non-null    int64  
 2   AgencyCode             101 non-null    int64  
 3   AgencyName             101 non-null    object 
 4   EmployeeID             101 non-null    int64  
 5   LastName               101 non-null    object 
 6   FirstName              101 non-null    object 
 7   AgencyStartDate        101 non-null    object 
 8   WorkLocationBorough    101 non-null    object 
 9   TitleCode              101 non-null    int64  
 10  TitleDescription       101 non-null    object 
 11  LeaveStatusasofJune30  101 non-null    object 
 12  BaseSalary             101 non-null    float64
 13  PayBasis               101 non-null    object 
 14  RegularHours           101 non-null    int64  
 15  Regula

In [12]:
 # Converting the FiscalYear to datetime
nycpayroll2021_df['FiscalYear'] = pd.to_datetime(nycpayroll2021_df['FiscalYear'], format='%Y') 

In [13]:
nycpayroll2021_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   FiscalYear             101 non-null    datetime64[ns]
 1   PayrollNumber          101 non-null    int64         
 2   AgencyCode             101 non-null    int64         
 3   AgencyName             101 non-null    object        
 4   EmployeeID             101 non-null    int64         
 5   LastName               101 non-null    object        
 6   FirstName              101 non-null    object        
 7   AgencyStartDate        101 non-null    object        
 8   WorkLocationBorough    101 non-null    object        
 9   TitleCode              101 non-null    int64         
 10  TitleDescription       101 non-null    object        
 11  LeaveStatusasofJune30  101 non-null    object        
 12  BaseSalary             101 non-null    float64       
 13  PayBa

In [15]:
agency_df = pd.read_csv(r'Data\AgencyMaster.csv')

In [16]:
agency_df.head()

,AgencyID,AgencyName
0,2001,ADMIN FOR CHILDREN'S SVCS
1,2002,ADMIN TRIALS AND HEARINGS
2,2003,BOARD OF CORRECTION
3,2004,BOARD OF ELECTION
4,2005,BOARD OF ELECTION POLL WORKERS


In [17]:
agency_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   AgencyID    153 non-null    int64 
 1   AgencyName  153 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.5+ KB


In [18]:
agency_df.shape

(153, 2)

In [19]:
employee_df = pd.read_csv(r'Data\EmpMaster.csv')

In [20]:
employee_df.head()

,EmployeeID,LastName,FirstName
0,100001,AACHEN,DAVID
1,100002,AACHEN,MONICA
2,100003,AADAMS,LAMMELL
3,100004,AADIL,IRIS
4,100005,AALAAM,AMIR


In [21]:
employee_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   EmployeeID  1000 non-null   int64 
 1   LastName    1000 non-null   object
 2   FirstName   1000 non-null   object
dtypes: int64(1), object(2)
memory usage: 23.6+ KB


In [22]:
nycpayroll2020_df.columns

Index(['FiscalYear', 'PayrollNumber', 'AgencyID', 'AgencyName', 'EmployeeID',
       'LastName', 'FirstName', 'AgencyStartDate', 'WorkLocationBorough',
       'TitleCode', 'TitleDescription', 'LeaveStatusasofJune30', 'BaseSalary',
       'PayBasis', 'RegularHours', 'RegularGrossPaid', 'OTHours',
       'TotalOTPaid', 'TotalOtherPay'],
      dtype='object')